In [21]:
import os, sys
import numpy as np
from collections import OrderedDict as OD
#import uproot3 as uproot
#import uproot3 
import uproot as uproot
import hist
import matplotlib.pyplot as plt
import mplhep as hep
import json

sys.path.insert(1, '../') # to import file from other directory (../ in this case)

from htoaa_CommonTools import(
    getTH1BinContent
)

#sIpFile = '/eos/cms/store/user/ssawant/htoaa/analysis/20230707_DataVsMC_HEM1516IssueDownwgt_woBtagWgt/2018/analyze_htoaa_stage1.root'
#sOpDir  = '/eos/cms/store/user/ssawant/htoaa/analysis/20230707_DataVsMC_HEM1516IssueDownwgt_woBtagWgt/2018/plots'
sIpFile = '/eos/cms/store/user/ssawant/htoaa/analysis/20230720_DataVsMC_woBtagWgt_wDeepTagbbVsL/2018/analyze_htoaa_stage1.root'
sOpDir  = '/eos/cms/store/user/ssawant/htoaa/analysis/20230720_DataVsMC_woBtagWgt_wDeepTagbbVsL/2018/plots'



if not os.path.exists(sOpDir):
    os.makedirs(sOpDir)
    
fIpFile = uproot.open(sIpFile)

In [22]:
sData = "Data"
MCBkg_list = ['QCD_bEnrich', 'QCD_bGen', 'QCD_Incl', 'TTJets_NLO', 'WJetsToQQ_HT', 'ZJetsToQQ_HT']
MCSig_list = ['SUSY_GluGluH_01J_HToAATo4B_M-20_HPtAbv150']
systematics_list = ['central']
systematics_forData = 'noweight'
xValueForEventYield = 4

eventYields = OD()
sEventYield = "%s \n" % (sIpFile)
MCBkg_total = 0
MCSig_total = 0
for systematics_name in systematics_list:
    sEventYield += "Systematics: %s \n" % (systematics_name)
    sEventYield += "%-45s:  %15s +- %15s  (%15s) \n" % ('Samples', 'nEvents', 'Uncertainty', 'nEvents_unweighted')
    eventYields[systematics_name] = OD()
    MCBkg_total = 0
    MCSig_total = 0
    for sample_category in [sData] + MCBkg_list + MCSig_list:
        # "hCutFlowWeighted"
        systematics_name_toUse = systematics_name if sample_category is not sData else systematics_forData

        histo_name_full = 'evt/%s/%s_%s' % (sample_category, "hCutFlowWeighted", systematics_name_toUse)
        h = fIpFile[histo_name_full].to_hist()
        nEvt_and_var = getTH1BinContent(h, xValue=xValueForEventYield)

        histo_name_full = 'evt/%s/%s_%s' % (sample_category, "hCutFlow", systematics_name_toUse)
        h = fIpFile[histo_name_full].to_hist()
        nEvt_and_var_unwgt = getTH1BinContent(h, xValue=xValueForEventYield)


        #print(f"{sample_category = },  \t\t\t\t\t nEvents: {nEvt_and_var.value}, \t\t  {np.sqrt(nEvt_and_var.variance)}")
        #print("%-45s:  %15g +- %15g  (%15d)" % (sample_category, nEvt_and_var.value, np.sqrt(nEvt_and_var.variance), nEvt_and_var_unwgt.value))
        sEventYield += "%-45s:  %15g +- %15g  (%15d) \n" % (sample_category, nEvt_and_var.value, np.sqrt(nEvt_and_var.variance), nEvt_and_var_unwgt.value)

        if sample_category in MCBkg_list:      MCBkg_total += nEvt_and_var.value
        if sample_category in MCSig_list:      MCSig_total += nEvt_and_var.value

        eventYields[systematics_name][sample_category] = OD([
            ('nEvents', nEvt_and_var.value),
            ('Uncertainty', np.sqrt(nEvt_and_var.variance)),
            ('nEventsUnwgt', nEvt_and_var_unwgt.value),
            ])  
        
    sEventYield += "\n"
    sEventYield += "%-45s:  %15g  \n" % ('Bkg_total', MCBkg_total)
    sEventYield += "%-45s:  %15g  \n" % ('S/sqrt(B)', MCSig_total / np.sqrt(MCBkg_total))
    sEventYield += "%-45s:  %15g  \n" % ('Data/MCBkg', eventYields[systematics_name][sData]['nEvents'] / MCBkg_total)

print("%s" % (sEventYield))
#print(f"{json.dumps(eventYields, indent=4) = }")
#json.dumps(eventYields, indent=4)

with open('%s/eventYieldTable.txt' % (sOpDir), 'w') as fOut:
    fOut.write(sEventYield)

/eos/cms/store/user/ssawant/htoaa/analysis/20230720_DataVsMC_woBtagWgt_wDeepTagbbVsL/2018/analyze_htoaa_stage1.root 
Systematics: central 
Samples                                      :          nEvents +-     Uncertainty  (nEvents_unweighted) 
Data                                         :           440412 +-         663.635  (         440412) 
QCD_bEnrich                                  :           124395 +-         640.227  (         102510) 
QCD_bGen                                     :           102390 +-         601.232  (          57577) 
QCD_Incl                                     :           257549 +-         1065.08  (         112861) 
TTJets_NLO                                   :          34462.1 +-         218.094  (         291183) 
WJetsToQQ_HT                                 :          1954.35 +-         15.9033  (          16543) 
ZJetsToQQ_HT                                 :          7567.26 +-         24.0451  (         103377) 
SUSY_GluGluH_01J_HToAATo4B_M-20_HP